In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-data-price-prediction/data_after_eda_Bivariate_and_multivariate_analysis


In [1]:

## import library
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import re
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import ExtraTreesRegressor ,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from skopt import BayesSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
from skopt.callbacks import DeadlineStopper
from cuml.ensemble import RandomForestRegressor as cumlRF
import optuna

In [2]:
df=pd.read_csv('/kaggle/input/flight-data-price-prediction/data_after_eda_Bivariate_and_multivariate_analysis')

In [3]:
df.head()

,flight_month,is_hoiliday,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration,price,Stops,price_category
0,6,0,0,SpiceJet,economy,Delhi,1135,Mumbai,1265,130,6013,0,Medium
1,6,0,0,SpiceJet,economy,Delhi,380,Mumbai,520,140,6013,0,Medium
2,6,0,0,AirAsia,economy,Delhi,265,Mumbai,395,130,6016,0,Medium
3,6,0,0,Vistara,economy,Delhi,620,Mumbai,755,135,6015,0,Medium
4,6,0,0,Vistara,economy,Delhi,530,Mumbai,670,140,6015,0,Medium


In [4]:
df=df.drop(columns=['is_hoiliday','Stops','price_category'])
df.head()

,flight_month,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration,price
0,6,0,SpiceJet,economy,Delhi,1135,Mumbai,1265,130,6013
1,6,0,SpiceJet,economy,Delhi,380,Mumbai,520,140,6013
2,6,0,AirAsia,economy,Delhi,265,Mumbai,395,130,6016
3,6,0,Vistara,economy,Delhi,620,Mumbai,755,135,6015
4,6,0,Vistara,economy,Delhi,530,Mumbai,670,140,6015


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298133 entries, 0 to 298132
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   flight_month    298133 non-null  int64 
 1   day_of_week     298133 non-null  int64 
 2   airline         298133 non-null  object
 3   class           298133 non-null  object
 4   departure_city  298133 non-null  object
 5   departure_time  298133 non-null  int64 
 6   arrival_city    298133 non-null  object
 7   arrival_time    298133 non-null  int64 
 8   duration        298133 non-null  int64 
 9   price           298133 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 22.7+ MB


In [6]:
X=df.drop(columns='price')
y=df['price']

In [8]:
X.shape

(298133, 9)

In [8]:
y.shape

(298133,)

In [26]:
X.columns

Index(['flight_month', 'day_of_week', 'airline', 'class', 'departure_city',
       'departure_time', 'arrival_city', 'arrival_time', 'duration'],
      dtype='object')

# `Check waether month or day_of_week can used as num or cat`

In [16]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Common features
cat_base = ['airline', 'class', 'departure_city', 'arrival_city']
num_base = ['day_of_week', 'departure_time', 'arrival_time', 'duration']

In [30]:
num_features_v1 = num_base + ['flight_month']
cat_features_v1 = cat_base

preprocessor_v1 = ColumnTransformer([
    ('num', StandardScaler(), num_features_v1),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features_v1)
])

model_v1 = Pipeline([
    ('prep', preprocessor_v1),
    ('model', LGBMRegressor(device='gpu', random_state=42))
])

model_v1.fit(X_train, y_train)
train_pred_v1 = model_v1.predict(X_train)
test_pred_v1 = model_v1.predict(X_test)

mae_train_v1 = mean_absolute_error(y_train, train_pred_v1)
mae_test_v1 = mean_absolute_error(y_test, test_pred_v1)


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 762
[LightGBM] [Info] Number of data points in the train set: 238506, number of used features: 27
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (2.73 MB) transferred to GPU in 0.005869 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 21191.536175


In [31]:
num_features_v2 = num_base
cat_features_v2 = cat_base + ['flight_month']

preprocessor_v2 = ColumnTransformer([
    ('num', StandardScaler(), num_features_v2),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features_v2)
])

model_v2 = Pipeline([
    ('prep', preprocessor_v2),
    ('model', LGBMRegressor(device='gpu', random_state=42,verbose=-1))
])

model_v2.fit(X_train, y_train)
train_pred_v2 = model_v2.predict(X_train)
test_pred_v2 = model_v2.predict(X_test)

mae_train_v2 = mean_absolute_error(y_train, train_pred_v2)
mae_test_v2 = mean_absolute_error(y_test, test_pred_v2)

In [32]:
print("🔹 Flight_Month as NUM:")
print(f"Train MAE: {mae_train_v1:.2f}")
print(f"Test  MAE: {mae_test_v1:.2f}\n")

print("🔸 Flight_Month as CAT:")
print(f"Train MAE: {mae_train_v2:.2f}")
print(f"Test  MAE: {mae_test_v2:.2f}")

🔹 Flight_Month as NUM:
Train MAE: 2562.43
Test  MAE: 2572.95

🔸 Flight_Month as CAT:
Train MAE: 2562.43
Test  MAE: 2572.94


In [33]:
# Common features
cat_base = ['airline', 'class', 'departure_city', 'arrival_city']
num_base = [ 'departure_time', 'arrival_time', 'duration','flight_month']

In [34]:
num_features_v1 = num_base + ['day_of_week']
cat_features_v1 = cat_base

preprocessor_v1 = ColumnTransformer([
    ('num', StandardScaler(), num_features_v1),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features_v1)
])

model_v1 = Pipeline([
    ('prep', preprocessor_v1),
    ('model', LGBMRegressor(device='gpu', random_state=42,verbose=-1))
])

model_v1.fit(X_train, y_train)
train_pred_v1 = model_v1.predict(X_train)
test_pred_v1 = model_v1.predict(X_test)

mae_train_v1 = mean_absolute_error(y_train, train_pred_v1)
mae_test_v1 = mean_absolute_error(y_test, test_pred_v1)


In [35]:
num_features_v1 = num_base 
cat_features_v1 = cat_base + ['day_of_week']

preprocessor_v1 = ColumnTransformer([
    ('num', StandardScaler(), num_features_v1),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features_v1)
])

model_v1 = Pipeline([
    ('prep', preprocessor_v1),
    ('model', LGBMRegressor(device='gpu', random_state=42))
])

model_v1.fit(X_train, y_train)
train_pred_v1 = model_v1.predict(X_train)
test_pred_v1 = model_v1.predict(X_test)

mae_train_v1 = mean_absolute_error(y_train, train_pred_v1)
mae_test_v1 = mean_absolute_error(y_test, test_pred_v1)


In [36]:
print("🔹 day_of_week as NUM:")
print(f"Train MAE: {mae_train_v1:.2f}")
print(f"Test  MAE: {mae_test_v1:.2f}\n")

print("🔸 day_of_week as CAT:")
print(f"Train MAE: {mae_train_v2:.2f}")
print(f"Test  MAE: {mae_test_v2:.2f}")

🔹 day_of_week as NUM:
Train MAE: 2558.76
Test  MAE: 2571.28

🔸 day_of_week as CAT:
Train MAE: 2562.43
Test  MAE: 2572.94


In [ ]:
numerical_columns=['flight_month', 'day_of_week','departure_time', 'arrival_time', 'duration']
# Preprocessor
preprocessor1 = ColumnTransformer([
    ('num', StandardScaler(), numerical_columns),('cat',OrdinalEncoder(),columns_to_encode),
    ('cat1', OneHotEncoder(drop='first'), categorical_columns)
])

# `Transformation`

In [7]:
# Common features
cat_base = ['airline', 'class', 'departure_city', 'arrival_city']
num_base = [ 'departure_time', 'arrival_time', 'duration','flight_month','day_of_week']

In [8]:
preprocessor1 = ColumnTransformer([
    ('num', StandardScaler(), num_base),
    ('cat1', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_base)
])

In [12]:
pipeline=Pipeline([
        ('preprocessor', preprocessor1),
        ('regressor', LinearRegression())
    ])

In [13]:
kfold1 = KFold(n_splits=25, shuffle=True, random_state=42)
# Perform cross-validation and calculate the scores
cv_scores = cross_val_score(pipeline,X,y,cv=kfold1, scoring='r2',error_score='raise')
print('training_data',np.mean(cv_scores)*100)
print('training_data',np.std(cv_scores))

training_data 90.1686584155006
training_data 0.002196760999649877


In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['departure_time',
                                                   'arrival_time', 'duration',
                                                   'flight_month',
                                                   'day_of_week']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['airline', 'class',
                                                   'departure_city',
                                                   'arrival_city'])])),
                ('regressor', LinearRegression())])

In [16]:
y_pred=pipeline.predict(X_test)

In [17]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error',mean_absolute_error(y_test,y_pred))

mean_absolute_error 4646.418205920345


# `ONE HOT ENCODING `

In [21]:
models = {
    'linear_reg': LinearRegression(),
    'decision tree': DecisionTreeRegressor(),
    'random forest': RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),'LGB':LGBMRegressor(),'XGB':XGBRegressor()}

In [22]:
def build_pipeline(model):
    return Pipeline([
        ('preprocessor', preprocessor1),
        ('regressor', model)
    ])

In [23]:
def evaluate_model(model_name, model):
    pipeline = build_pipeline(model)
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    # Cross-validation
    cv_scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='r2', error_score='raise')

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_pred_actual = y_pred
    y_test_actual = y_test

    # Metrics
    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
    r2 = r2_score(y_test_actual, y_pred_actual)

    print(f"{model_name} model training and evaluation completed.")

    return {
        'Model': model_name,
        'CV_R2': cv_scores.mean(),
        'MAE': mae,
        'RMSE': rmse,
        'Test_R2': r2
    }


In [26]:
results = []
for model_name, model in models.items():
    result = evaluate_model(model_name, model)
    results.append(result)

results_df = pd.DataFrame(results).sort_values(by='Test_R2', ascending=False)

In [25]:
results_df = pd.DataFrame(results).sort_values(by=['MAE','Test_R2'], ascending=[True,False])
results_df

,Model,CV_R2,MAE,RMSE,Test_R2
2,random forest,0.979163,1876.346387,3349.182019,0.978782
3,extra trees,0.977926,1898.800818,3483.221077,0.977050
1,decision tree,0.975697,1936.632824,3677.662824,0.974416
4,gradient boosting,0.949534,3329.244348,5139.889915,0.950028
5,adaboost,0.929281,4231.132977,6153.274122,0.928381
0,linear_reg,0.901692,4646.418206,7200.161492,0.901938


In [38]:
models = {'XGB':XGBRegressor(
    tree_method='hist',
    device='cuda',  # replaces tree_method='gpu_hist' and gpu_id
    verbosity=1
),'LGB':LGBMRegressor(
    device='gpu',
    gpu_use_dp=True,  # or False for single precision (faster, less accurate)
    boosting_type='gbdt'
)}

In [41]:
def evaluate_model(model_name, model):
    pipeline = build_pipeline(model)
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    # Cross-validation
    cv_scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='r2', error_score='raise')

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_pred_actual = y_pred
    y_test_actual = y_test

    # Metrics
    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
    r2 = r2_score(y_test_actual, y_pred_actual)

    print(f"{model_name} model training and evaluation completed.")

    return {
        'Model': model_name,
        'CV_R2': cv_scores.mean(),
        'MAE': mae,
        'RMSE': rmse,
        'Test_R2': r2
    }


In [42]:
results = []
for model_name, model in models.items():
    result = evaluate_model(model_name, model)
    results.append(result)

XGB model training and evaluation completed.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 753
[LightGBM] [Info] Number of data points in the train set: 268319, number of used features: 23
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 9 dense feature groups (3.07 MB) transferred to GPU in 0.006518 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 21178.664306
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 754
[LightGBM] [Info] Number of data points in the train set: 268319, number of used features: 23
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram 

In [43]:
results_df1 = pd.DataFrame(results).sort_values(by=['MAE','Test_R2'], ascending=[True,False])
results_df1

,Model,CV_R2,MAE,RMSE,Test_R2
0,XGB,0.977904,2219.126147,3400.528396,0.978127
1,LGB,0.971749,2584.962899,3873.841229,0.971614


In [48]:
result=pd.concat([results_df,results_df1])

In [49]:
result.sort_values(by=['MAE','Test_R2'], ascending=[True,False])
result

,Model,CV_R2,MAE,RMSE,Test_R2
2,random forest,0.979163,1876.346387,3349.182019,0.978782
3,extra trees,0.977926,1898.800818,3483.221077,0.977050
1,decision tree,0.975697,1936.632824,3677.662824,0.974416
4,gradient boosting,0.949534,3329.244348,5139.889915,0.950028
5,adaboost,0.929281,4231.132977,6153.274122,0.928381
0,linear_reg,0.901692,4646.418206,7200.161492,0.901938
0,XGB,0.977904,2219.126147,3400.528396,0.978127
1,LGB,0.971749,2584.962899,3873.841229,0.971614


# `Random forest and XGB ml algo. which is giving  good result`

In [59]:
pipeline=Pipeline([
        ('preprocessor', preprocessor1),
        ('regressor', RandomForestRegressor())])

In [72]:
# Preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_base),
    ('cat1', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_base)
])

X_preprocessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

def objective(trial):
    model = cumlRF(
        n_estimators=trial.suggest_int('n_estimators', 100, 500),
        max_features=trial.suggest_float('max_features', 0.3, 1.0),
        max_depth=trial.suggest_int('max_depth', 5, 40),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
        bootstrap=True,
        n_streams=8  # number of parallel threads (optional)
    )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return r2_score(y_test, preds)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Final model with best parameters
best_params = study.best_params
final_model = cumlRF(**best_params)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print(f"R-squared Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")

[I 2025-04-06 16:53:45,013] A new study created in memory with name: no-name-72232d88-a56f-4de4-8565-d619f08093ef
/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
[I 2025-04-06 16:54:15,084] Trial 0 finished with value: 0.9780058817149788 and parameters: {'n_estimators': 415, 'max_features': 0.7803902941278476, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9780058817149788.
/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
[I 2025-04-06 16:54:41,576] Trial 1 finished with value: 0.9812863090617768 and parameters: {'n_estimators': 312, 'max_features': 0.7337155757492693, 'max_depth': 23, 'min_samples_split': 15, 'min_samples_leaf': 4}. Best is trial 1 with value: 0

Best Hyperparameters: {'n_estimators': 170, 'max_features': 0.8064390412828828, 'max_depth': 23, 'min_samples_split': 18, 'min_samples_leaf': 3}
R-squared Score: 0.9814
MAE: 1859.6687


In [78]:
pipeline=Pipeline([
        ('preprocessor', preprocessor1),
        ('regressor', XGBRegressor(
    tree_method='hist',
    device='cuda',
    random_state=42
))])

In [80]:
# Preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_base),
    ('cat1', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_base)
])

X_preprocessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Optuna Objective
def objective(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 100, 500),
        max_depth=trial.suggest_int('max_depth', 3, 15),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        subsample=trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.3, 1.0),
        gamma=trial.suggest_float('gamma', 0.0, 5.0),
        reg_alpha=trial.suggest_float('reg_alpha', 0.0, 5.0),
        reg_lambda=trial.suggest_float('reg_lambda', 0.0, 5.0),
        tree_method='hist',
        device='cuda',
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return r2_score(y_test, preds)

# Run Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

# Best model
best_params = study.best_params
final_model = XGBRegressor(
    **best_params,
    tree_method='hist',
    device='cuda',
    random_state=42
)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print(f"R-squared Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")

[I 2025-04-06 18:01:18,100] A new study created in memory with name: no-name-74b41688-ddbf-4794-bb41-4b79fb806474
[I 2025-04-06 18:01:20,437] Trial 0 finished with value: 0.9833802924689801 and parameters: {'n_estimators': 216, 'max_depth': 9, 'learning_rate': 0.228981800894394, 'subsample': 0.9340162063030981, 'colsample_bytree': 0.5715103136594519, 'gamma': 1.4899733642461965, 'reg_alpha': 4.278877941396562, 'reg_lambda': 0.6674708343530267}. Best is trial 0 with value: 0.9833802924689801.
[I 2025-04-06 18:01:22,505] Trial 1 finished with value: 0.9607866037442498 and parameters: {'n_estimators': 331, 'max_depth': 7, 'learning_rate': 0.015670124393885085, 'subsample': 0.6659260110258229, 'colsample_bytree': 0.5843366677084597, 'gamma': 3.716981873737949, 'reg_alpha': 1.8955474961419305, 'reg_lambda': 4.5504232159852265}. Best is trial 0 with value: 0.9833802924689801.
[I 2025-04-06 18:01:24,324] Trial 2 finished with value: 0.9730854311676064 and parameters: {'n_estimators': 200, 'ma

Best Hyperparameters: {'n_estimators': 429, 'max_depth': 9, 'learning_rate': 0.18772568687150198, 'subsample': 0.9406102768735505, 'colsample_bytree': 0.6819793543212364, 'gamma': 1.6859685015180228, 'reg_alpha': 3.9907215626899215, 'reg_lambda': 1.396751406385936}
R-squared Score: 0.9839
MAE: 1796.9637


In [81]:
# Preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_base),
    ('cat1', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_base)
])

X_preprocessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Optuna Objective
def objective(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 100, 500),
        max_depth=trial.suggest_int('max_depth', 3, 15),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        subsample=trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.3, 1.0),
        gamma=trial.suggest_float('gamma', 0.0, 5.0),
        reg_alpha=trial.suggest_float('reg_alpha', 0.0, 5.0),
        reg_lambda=trial.suggest_float('reg_lambda', 0.0, 5.0),
        tree_method='hist',
        device='cuda',
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return r2_score(y_test, preds)

# Run Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

# Best model
best_params_500 = study.best_params
final_model = XGBRegressor(
    **best_params_500,
    tree_method='hist',
    device='cuda',
    random_state=42
)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Best Hyperparameters:", best_params_500)
print(f"R-squared Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")


[I 2025-04-06 18:09:30,301] A new study created in memory with name: no-name-69ecadc2-618d-4a57-b391-2ddb5a8373d0
[I 2025-04-06 18:09:36,803] Trial 0 finished with value: 0.9783834547768165 and parameters: {'n_estimators': 209, 'max_depth': 14, 'learning_rate': 0.0489169597875383, 'subsample': 0.7912749826083383, 'colsample_bytree': 0.40514685359967983, 'gamma': 4.23845237140666, 'reg_alpha': 4.372124845824386, 'reg_lambda': 4.712425519730372}. Best is trial 0 with value: 0.9783834547768165.
[I 2025-04-06 18:09:38,223] Trial 1 finished with value: 0.9794828426909866 and parameters: {'n_estimators': 125, 'max_depth': 9, 'learning_rate': 0.10496640347188763, 'subsample': 0.9258454303106569, 'colsample_bytree': 0.66854704721919, 'gamma': 2.568535987693717, 'reg_alpha': 4.808732627790395, 'reg_lambda': 3.3244601485392593}. Best is trial 1 with value: 0.9794828426909866.
[I 2025-04-06 18:09:39,353] Trial 2 finished with value: 0.9541790651076074 and parameters: {'n_estimators': 338, 'max_de

Best Hyperparameters: {'n_estimators': 457, 'max_depth': 9, 'learning_rate': 0.11132706898446479, 'subsample': 0.9795941409437815, 'colsample_bytree': 0.9516371445970983, 'gamma': 3.813583010719894, 'reg_alpha': 3.76450269038949, 'reg_lambda': 3.953492916515455}
R-squared Score: 0.9844
MAE: 1765.3403


In [82]:
best_params_500

{'n_estimators': 457,
 'max_depth': 9,
 'learning_rate': 0.11132706898446479,
 'subsample': 0.9795941409437815,
 'colsample_bytree': 0.9516371445970983,
 'gamma': 3.813583010719894,
 'reg_alpha': 3.76450269038949,
 'reg_lambda': 3.953492916515455}

In [107]:
pipeline=Pipeline([
        ('preprocessor', preprocessor1),
        ('regressor', final_model)])

In [ ]:
final_model

In [108]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [109]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['departure_time',
                                                   'arrival_time', 'duration',
                                                   'flight_month',
                                                   'day_of_week']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['airline', 'class',
                                                   'departure_city',
                                                   'arrival_city'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              c...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.11132706898446479, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=9, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=457, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [110]:
y_pred=pipeline.predict(X_test)
print(r2_score(y_test,y_pred))

0.9877813416415632


In [111]:

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_train = cross_val_score(
    pipeline, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error'
)
print("Mean Absolute Error (Train):", -np.mean(cv_scores_train))

cv_scores_test = cross_val_score(
    pipeline, X_test, y_test, cv=kfold, scoring='neg_mean_absolute_error'
)
print("Mean Absolute Error (Test):", -np.mean(cv_scores_test))
cv_scores_test = cross_val_score(
    pipeline, X, y, cv=kfold, scoring='neg_mean_absolute_error'
)
print("Mean Absolute Error (Test):", -np.mean(cv_scores_test))

Mean Absolute Error (Train): 1786.6385262869753
Mean Absolute Error (Test): 2001.704444556604
Mean Absolute Error (Test): 1758.929918618544


In [112]:
kfold = KFold(n_splits=25, shuffle=True, random_state=42)
cv_scores_train = cross_val_score(
    pipeline, X_train, y_train, cv=kfold, scoring='r2'
)
print("r2 (Train):",np.mean(cv_scores_train))

cv_scores_test = cross_val_score(
    pipeline, X_test, y_test, cv=kfold, scoring='r2'
)
print("r2 (Test):", np.mean(cv_scores_test))

r2 (Train): 0.984155384818235
r2 (Test): 0.9801348210802244


In [113]:
with open('model(XGB).pikle','wb') as file:
  pickle.dump(pipeline,file)

In [114]:
X

,flight_month,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration
0,6,0,SpiceJet,economy,Delhi,1135,Mumbai,1265,130
1,6,0,SpiceJet,economy,Delhi,380,Mumbai,520,140
2,6,0,AirAsia,economy,Delhi,265,Mumbai,395,130
3,6,0,Vistara,economy,Delhi,620,Mumbai,755,135
4,6,0,Vistara,economy,Delhi,530,Mumbai,670,140
...,...,...,...,...,...,...,...,...,...
298128,8,6,Vistara,business,Chennai,585,Hyderabad,1190,605
298129,8,6,Vistara,business,Chennai,750,Hyderabad,1375,625
298130,8,6,Vistara,business,Chennai,425,Hyderabad,1255,830
298131,8,6,Vistara,business,Chennai,420,Hyderabad,1020,600


In [115]:
with open('data.df(1)','wb') as file:
  pickle.dump(X,file)

In [116]:
data=[[12,2,'Vistara','business','Chennai',585,'Hyderabad',119,180]]
columns=['flight_month','day_of_week','airline','class','departure_city','departure_time','arrival_city','arrival_time','duration']

In [117]:
# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)
one_df

,flight_month,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration
0,12,2,Vistara,business,Chennai,585,Hyderabad,119,180


In [118]:
pipeline.predict(one_df)

array([23413.902], dtype=float32)

In [12]:
# Preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_base),
    ('cat1', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_base)
])

X_preprocessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Optuna Objective
def objective(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 100, 500),
        max_depth=trial.suggest_int('max_depth', 3, 15),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        subsample=trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.3, 1.0),
        gamma=trial.suggest_float('gamma', 0.0, 5.0),
        reg_alpha=trial.suggest_float('reg_alpha', 0.0, 5.0),
        reg_lambda=trial.suggest_float('reg_lambda', 0.0, 5.0),
        tree_method='hist',
        device='cuda',
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return r2_score(y_test, preds)

# Run Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

# Best model
best_params_1000_xg = study.best_params
final_model_xgb = XGBRegressor(
    **best_params_500,
    tree_method='hist',
    device='cuda',
    random_state=42
)
final_model_xgb.fit(X_train, y_train)
y_pred = final_model_xgb.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Best Hyperparameters:", best_params_500)
print(f"R-squared Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")

[I 2025-04-07 06:39:21,859] A new study created in memory with name: no-name-a2066278-15f4-4189-ab0b-2c3162a9fffb
[I 2025-04-07 06:39:29,810] Trial 0 finished with value: 0.9815772281330456 and parameters: {'n_estimators': 415, 'max_depth': 11, 'learning_rate': 0.21547784819723015, 'subsample': 0.5960591169863532, 'colsample_bytree': 0.6464612105404308, 'gamma': 3.28463943519505, 'reg_alpha': 1.6475585110121604, 'reg_lambda': 1.1221719906855776}. Best is trial 0 with value: 0.9815772281330456.
[I 2025-04-07 06:39:31,099] Trial 1 finished with value: 0.9689180494167952 and parameters: {'n_estimators': 387, 'max_depth': 4, 'learning_rate': 0.12390334504834974, 'subsample': 0.9426868316898991, 'colsample_bytree': 0.8237417835735592, 'gamma': 0.3972115401264753, 'reg_alpha': 2.084115949206282, 'reg_lambda': 1.5257274992787244}. Best is trial 0 with value: 0.9815772281330456.
[I 2025-04-07 06:39:37,157] Trial 2 finished with value: 0.9825751313039572 and parameters: {'n_estimators': 487, 'm

NameError: name 'best_params_500' is not defined

In [17]:
pipeline = Pipeline([
    ('preprocessor', preprocessor1),('regressor',XGBRegressor(
    n_estimators=487,
    max_depth=8,
    learning_rate=0.18256206516894752,
    subsample=0.9998503269642816,
    colsample_bytree=0.9178089695039398,
    gamma=1.4095437905078125,
    reg_alpha=3.655344573366067,
    reg_lambda=2.235900419393253,
    tree_method='hist',
    device='cuda',
    random_state=42
))
])

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [19]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['departure_time',
                                                   'arrival_time', 'duration',
                                                   'flight_month',
                                                   'day_of_week']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['airline', 'class',
                                                   'departure_city',
                                                   'arrival_city'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              c...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.18256206516894752, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=8, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=487, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [20]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_train = cross_val_score(
    pipeline, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error'
)
print("Mean Absolute Error (Train):", -np.mean(cv_scores_train))

cv_scores_test = cross_val_score(
    pipeline, X_test, y_test, cv=kfold, scoring='neg_mean_absolute_error'
)
print("Mean Absolute Error (Test):", -np.mean(cv_scores_test))

Mean Absolute Error (Train): 1800.195933952567
Mean Absolute Error (Test): 2033.1574095906622


In [21]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_train = cross_val_score(
    pipeline, X_train, y_train, cv=kfold, scoring='r2'
)
print("r2 (Train):", np.mean(cv_scores_train))

cv_scores_test = cross_val_score(
    pipeline, X_test, y_test, cv=kfold, scoring='r2'
)
print("r2 (Test):", np.mean(cv_scores_test))

r2 (Train): 0.983990673177973
r2 (Test): 0.979551808869917


In [22]:
with open('model(RFR).pikle','wb') as file:
  pickle.dump(pipeline,file)

In [23]:
data = [[12, 2, 'Vistara', 'business', 'Chennai', 585, 'Hyderabad', 119, 180]]
columns = ['flight_month','day_of_week','airline','class','departure_city','departure_time','arrival_city','arrival_time','duration']


In [24]:
# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)
one_df

,flight_month,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration
0,12,2,Vistara,business,Chennai,585,Hyderabad,119,180


In [25]:
pipeline.predict(one_df)

array([21862.06], dtype=float32)